### Using S3 and Analytics

In this notebook, you will use S3 to download an portion of Amazon product reviews from our [Registry of Open Data](https://registry.opendata.aws/), use [Pandas](https://pandas.pydata.org/) to clean, transform, and export the reviews to prepare the data to train a sentiment analysis algorithm. Finally, you will be introduced to AWS Glue and Athena to catalog the metadata of the dataset and query it serverlessly using Athena. There is an optional section at the end to bake the ability to query with Athena into Jupyter using [custom magics](https://ipython.readthedocs.io/en/stable/config/custommagics.html) capabilities with Jupyter.

Update the variable **bucket** below with your initials in replace of the **{{FIXME}}** place holder.

In [ ]:
import boto3
import botocore
import json
import time
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown

glue = boto3.client('glue')
s3 = boto3.client('s3')
s3_resource = boto3.resource('s3')

session = boto3.session.Session()
region = session.region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')

bucket = 'escience-workshop-{{FIXME}}'
amazon_review_bucket = 'amazon-reviews-pds'

pd.set_option('display.max_colwidth', -1)
%matplotlib inline

### [Create S3 Bucket](https://docs.aws.amazon.com/AmazonS3/latest/gsg/CreatingABucket.html)

We will create an S3 bucket that will be used throughout the workshop for storing data.

[s3.create_bucket](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.create_bucket) boto3 documentation

In [ ]:
def create_bucket(bucket):
    import logging

    try:
        s3.create_bucket(Bucket=bucket, CreateBucketConfiguration={'LocationConstraint': region})
    except botocore.exceptions.ClientError as e:
        logging.error(e)
        return 'Bucket {0} could not be created.'.format(bucket)
    return 'Created {0} bucket.'.format(bucket)

In [ ]:
create_bucket(bucket)

### [Download Amazon Product Reviews](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/s3-example-download-file.html) 

We will download the reviews from the Amazon Product Review dataset available on the [AWS Open Data Registry](https://registry.opendata.aws/amazon-reviews/).

In [ ]:
try:
    s3_resource.Bucket(amazon_review_bucket).download_file('tsv/amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz', 'amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz')
except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == "404":
        print("The object does not exist.")
    else:
        raise

### Unzip Amazon Video Reviews



In [ ]:
!gzip -d amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz

In [ ]:
!ls -la

### View raw tsv file

We will use [Pandas](https://pandas.pydata.org/) [`read_csv`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) method to load the tsv into a [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html). The dataset contains a number of columns associated with each review. For this exercise you will only be concerned with the `review_body` and `star_rating` columns. The `star_rating` is between 1-5 and the `review` is a free form text field. You will then use the DataFrame's [`head()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.head.html) method to see the first few rows.

In [ ]:
df = pd.read_csv('amazon_reviews_us_Mobile_Electronics_v1_00.tsv', sep='\t', error_bad_lines=False)
df.head()

### View Data types of the columns in the data set

To see the data types of each column you can use the [`dtypes`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dtypes.html) function. Click [here](https://pandas.pydata.org/pandas-docs/stable/getting_started/basics.html#basics-dtypes) for more information on the basic dtypes supported.

In [ ]:
df.dtypes

### Create new DataFrame for only columns needed

Below we will will keep only the `review_body` and `star_rating` fields for our dataset. YOu can use the [loc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html) method to access just the columns and rows you need. You will also filter out the neutral reviews which we are signifying as reviews with a `3` star rating to make more definitive the positive vs. negative reviews.

In [ ]:
df = df.loc[:, ['review_body', 'star_rating']]
df = df.query('star_rating != 3.0')
df.head()

### Add new column `sentiment` based on `star_rating`

Next, we will use the [apply]() method to apply a function to each row in the DataFrame adding a new column `sentiment`. This will make it easier to visualize the data.

In [ ]:
def get_sentiment(value):
    if value > 3:
        return 1
    else:
        return 0

In [ ]:
df['sentiment'] = df['star_rating'].apply(get_sentiment)

In [ ]:
df.head()

### Basic Cleaning
There are a number of ways to clean the daatset you are working with. You will go through a few exercises looking at the data and perfoming actions based on the results to cleanse the data. The first thing you want to do is look at the [`unique`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.unique.html) `star_rating` values. You will notice there are reviews with a `nan` rating so we will want to remove those using the [`dropna`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html) method.

We can visualize the counts to quickly see the breakout by `star_rating`. For that you will be using [seaborn](https://seaborn.pydata.org/).

Next we want to **de-duplicate** the rows. It is necessary to remove duplicates in order to get unbaised results. Checking duplicates based on `review_body` and `star_rating`. If all values are equal then we will remove those records.

At this time we can also transform the 'review_body' from it's regular form, with capital letters to the pure lowercase text. We can also see below we have ratings with a `nan` value meaning no `star_rating` was giving for the review. We will drop the `nan` reviews as well to clean up the data set.

In [ ]:
df.star_rating.unique()

In [ ]:
df.dropna(inplace=True)

### Visualize the review counts

In [ ]:
print(df['star_rating'].value_counts())
sns.countplot(x='star_rating', data=df)

### Drop duplicates and set proper data types

In [ ]:
import string
df = df.drop_duplicates(subset={'review_body', 'star_rating'})

df.dropna(inplace=True)
df.loc[:, 'star_rating'] = df['star_rating'].astype(int)
df.loc[:, 'review_body'] = df['review_body'].astype(str).str.lower()
df.head()

In [ ]:
print(df['sentiment'].value_counts())
sns.countplot(x='sentiment', data=df)

### Python [Wordcloud](http://amueller.github.io/word_cloud/) Visualization

Another visualization technique you can use to see what words stand out in the data set is to build a Wordcloud visualization. First, we will use pip to install the library nad then build a simple function to plot the words.

In [ ]:
!python -m pip install wordcloud

In [ ]:
from wordcloud import WordCloud, STOPWORDS

def plot_wordcloud(text, mask=None, max_words=200, max_font_size=150, figure_size=(25.0,15.0), 
                   title = None, title_size=40, image_color=False):
    stopwords = set(STOPWORDS)

    wordcloud = WordCloud(background_color='gray',
                    stopwords = stopwords,
                    max_words = max_words,
                    max_font_size = max_font_size, 
                    random_state = 50,
                    width=800, 
                    height=400,
                    mask = mask)
    wordcloud.generate(str(text))
    
    plt.figure(figsize=figure_size)
    if image_color:
        image_colors = ImageColorGenerator(mask);
        plt.imshow(wordcloud.recolor(color_func=image_colors), interpolation="bilinear");
        plt.title(title, fontdict={'size': title_size,  
                                  'verticalalignment': 'bottom'})
    else:
        plt.imshow(wordcloud);
        plt.title(title, fontdict={'size': title_size, 'color': 'black', 
                                  'verticalalignment': 'bottom'})
    plt.axis('off');
    plt.tight_layout()  
    
plot_wordcloud(df['review_body'], title="Word Cloud of Amazon Reviews")

You can see in the wordcloud `br` which typically means there is HTML markeup in the `review_body`. We want to clean the reviews by removing the markup as well. We will use [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) to remove the markup. 

In [ ]:
import bs4
df['review_body'] = df['review_body'].apply(lambda x: bs4.BeautifulSoup(x, 'lxml').get_text())

### Discover 

In [ ]:
import string

## Number of words in the text ##
df["num_words"] = df["review_body"].apply(lambda x: len(str(x).split()))

## Number of unique words in the text ##
df["num_unique_words"] = df["review_body"].apply(lambda x: len(set(str(x).split())))

## Number of characters in the text ##
df["num_chars"] = df["review_body"].apply(lambda x: len(str(x)))

## Number of stopwords in the text ##
df["num_stopwords"] = df["review_body"].apply(lambda x: len([w for w in str(x).lower().split() if w in STOPWORDS]))

## Number of punctuations in the text ##
df["num_punctuations"] = df['review_body'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )

## Average length of the words in the text ##
df["mean_word_len"] = df["review_body"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

In [ ]:
df.describe()

In [ ]:
## Truncate some extreme values for better visuals, think about balancing positive vs negative reviews ##
df = df.query('num_words <= 500 and num_punctuations < 500')

In [ ]:
print(df['sentiment'].value_counts())
sns.countplot(x='sentiment', data=df)

In [ ]:
f, axes = plt.subplots(3, 1, figsize=(10,20))
sns.violinplot(x='sentiment', y='num_words', data=df, ax=axes[0])
axes[0].set_xlabel('Sentiment', fontsize=12)
axes[0].set_title("Number of words in each class", fontsize=15)

sns.violinplot(x='sentiment', y='num_chars', data=df, ax=axes[1])
axes[1].set_xlabel('Sentiment', fontsize=12)
axes[1].set_title("Number of characters in each class", fontsize=15)

sns.violinplot(x='sentiment', y='num_punctuations', data=df, ax=axes[2])
axes[2].set_xlabel('Sentiment', fontsize=12)
axes[2].set_title("Number of punctuations in each class", fontsize=15)
plt.show()

### Handling imbalanced datasets

Here you can see we have a larger number of `positive` samples vs. `negative` ones. There are a number of techniques to blance this dataset out and the two most popular approaches are to either under-sample or over-sample. With under sampling you remove rows to balance the dataset out and in over sampling you can duplicate entries in the daatset which could lead to overfitting. This discussion is beyond the scope of this lab. You will under sample the data to balance the dataset but you can find more information [here]().

In [ ]:
from sklearn.utils import resample

positive = df[df['sentiment']==1]
negative = df[df['sentiment']==0]

positive_downsampled = resample(positive,
                                replace = False, # sample without replacement
                                n_samples = len(negative), # match minority n
                                random_state = 27) # reproducible results

# combine minority and downsampled majority
downsampled = pd.concat([positive_downsampled, negative])

# checking counts
print(downsampled['sentiment'].value_counts())
sns.countplot(x='sentiment', data=downsampled)

### Export datasets

In [ ]:
import csv 

file_name = 'reviews.csv'
final_df = downsampled.loc[:, ['review_body', 'star_rating', 'sentiment']]
final_df['review_body'] = downsampled['review_body'].astype('str')
final_df.to_csv("reviews.csv", index=False, header=False, quoting=csv.QUOTE_ALL, escapechar='\\', doublequote=False)

### Unlabeled Data

We will also create a dataset here with the reviews only. This will simulate not having labelled data for training and in a follow up lab you will be able to use [SageMaker Ground Truth](https://aws.amazon.com/sagemaker/groundtruth/) to build up a labeled dataset for training.

In [ ]:
gt_df = downsampled.loc[:, ['review_body']]
gt_df.to_csv("groundtruth.csv", index=False, header=True)

In [ ]:
unlabeled_df = pd.read_csv('groundtruth.csv', error_bad_lines=False)
unlabeled_df.head()

### [Upload to S3](https://docs.aws.amazon.com/AmazonS3/latest/dev/Welcome.html)

Next, we will upload the files you created above to S3 to be used in a later workshop.

[s3.upload_file](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.upload_file) boto3 documentation

In [ ]:
s3_resource.Bucket(bucket).Object(os.path.join('raw', file_name)).upload_file(file_name)
s3_resource.Bucket(bucket).Object(os.path.join('unlabeled', 'groundtruth.csv')).upload_file('groundtruth.csv')

### List S3 Bucket Contents

Now that we have uploaded objects to our S3 bucket we can view the contents of the bucket as well through the SDK.

In [ ]:
paginator = s3.get_paginator('list_objects_v2')
for result in paginator.paginate(Bucket=bucket):
    if result.get('Contents') is not None:
        for files in result.get('Contents'):
            if not files.get('Key') == "":
                print(files.get('Key'))

### Create test, train, and validate datasets for training

Depending on the framework you are leveraging in your AI/ML workloads you may decide to split the data into test, train, and validate splits before uploading to S3. You can leverage some built in functions in the sklearn package to do the split. To learn more about the sklearn framework click [here](https://scikit-learn.org/stable/).

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(final_df, test_size=0.2, random_state=0)
train, validate = train_test_split(train, test_size=0.2, random_state=0)

print(f'Number of training examples: {len(train.index)}')
print(f'Number of validation examples: {len(validate.index)}')
print(f'Number of testing examples: {len(test.index)}')

In [ ]:
train.to_csv("train.csv", index=False, header=False, quoting=csv.QUOTE_NONE, escapechar='\\')
test.to_csv("test.csv", index=False, header=False, quoting=csv.QUOTE_NONE, escapechar='\\')
validate.to_csv("validate.csv", index=False, header=False, quoting=csv.QUOTE_NONE, escapechar='\\')

### [Upload to S3](https://docs.aws.amazon.com/AmazonS3/latest/dev/Welcome.html)

Now let's upload the splits created above to S3.

[s3.upload_file](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.upload_file) boto3 documentation

In [ ]:
s3_resource.Bucket(bucket).Object(os.path.join('data', 'train.csv')).upload_file('train.csv')
s3_resource.Bucket(bucket).Object(os.path.join('data', 'test.csv')).upload_file('test.csv')
s3_resource.Bucket(bucket).Object(os.path.join('data', 'validate.csv')).upload_file('validate.csv')

### Democratizing Data

You may determine that you want to make this dataset available to your lab or team to give them the ability to perform their own research on this same dataset. You can utilize a number of services in the [AWS Data Lakes & Analytics](https://aws.amazon.com/big-data/datalakes-and-analytics/) group to do so. This is the start of building out a [Data Lake on AWS](https://aws.amazon.com/big-data/datalakes-and-analytics/what-is-a-data-lake/?nc=sn&loc=2). We will start by cataloging the raw data in the Glue Data Catalog which is a Hive compliant metadata repository.


### Create the [AWS Glue Catalog Database](https://docs.aws.amazon.com/glue/latest/dg/define-database.html)

When you define a table in the AWS Glue Data Catalog, you add it to a database. A database is used to organize tables in AWS Glue. You can organize your tables using a crawler or using the AWS Glue console. A table can be in only one database at a time.

There is a central Glue Catalog for each AWS account. When creating the database you will use your account id declared above as `account_id`

[glue.create_database](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/glue.html#Glue.Client.create_database)

In [ ]:
def create_database(glue_client, account_id, database_name, description):
    """Create the specified Glue database if it does not exist"""
    try:
        glue_client.get_database(
            CatalogId=account_id,
            Name=database_name
        )
    except glue_client.exceptions.EntityNotFoundException:
        print("Creating database: %s" % database_name)
        glue_client.create_database(
            CatalogId=account_id,
            DatabaseInput={
                'Name': database_name,
                'Description': description
            }
        )

In [ ]:
database_name = 'reviews'
create_database(glue, account_id, database_name, 'Database for Amazon Reviews')

### [Create the Raw table in Glue](https://docs.aws.amazon.com/glue/latest/dg/tables-described.html)

When you define a table in AWS Glue, you also specify the value of a classification field that indicates the type and format of the data that's stored in that table. If a crawler creates the table, these classifications are determined by either a built-in classifier or a custom classifier. If you create a table manually in the console or by using an API, you specify the classification when you define the table. For more information about creating a table using the AWS Glue console, see [Working with Tables on the AWS Glue Console](https://docs.aws.amazon.com/glue/latest/dg/console-tables.html).

[glue.create_table](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/glue.html#Glue.Client.create_table)

In [ ]:
location = 's3://{0}/raw/'.format(bucket)
raw_table_name = 'raw'

response = glue.create_table(
    CatalogId=account_id,
    DatabaseName=database_name,
    TableInput={
        'Name': raw_table_name,
        'Description': 'Raw Amazon Mobile Products reviews',
        'StorageDescriptor': {
            'Columns': [                
                {
                    'Name': 'review_body',
                    'Type': 'string',
                    'Comment': 'Review text of from the review'
                },         
                {
                    'Name': 'star_rating',
                    'Type': 'float',
                    'Comment': 'Star rating of from the review'
                },
                {
                    'Name': 'sentiment',
                    'Type': 'tinyint',
                    'Comment': 'Sentiment based on the star rating of from the review'
                }
            ],
            'Location': location,
            'InputFormat': 'org.apache.hadoop.mapred.TextInputFormat',
            'OutputFormat': 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat',
            'SerdeInfo': {
                'SerializationLibrary': 'org.apache.hadoop.hive.serde2.OpenCSVSerde',
                'Parameters': {
                    'escapeChar': '\\',
                    'separatorChar': ',',
                    'serialization.format': '1'
                }
            },
        },
        'TableType': 'EXTERNAL_TABLE',
        'Parameters': {
            'classification': 'csv'
        }
    }
)

### [Query Raw Amazon Reviews with Athena ](https://docs.aws.amazon.com/athena/latest/ug/getting-started.html)

To see the raw reviews we will be installing a python library for querying the data in the Glue Data Catalog with Athena. More information about [PyAthena](https://pypi.org/project/PyAthena/) or available [ANSI SQL and DDL commands with Athena](https://docs.aws.amazon.com/athena/latest/ug/ddl-sql-reference.html)

In [ ]:
!pip install PyAthena

In [ ]:
from pyathena import connect
from pyathena.util import as_pandas

cursor = connect(region_name=region, s3_staging_dir='s3://{0}/temp'.format(bucket)).cursor()
cursor.execute('select * from {0}.{1} limit 10'.format(database_name, raw_table_name))

df = as_pandas(cursor)
df.head(5)

In [ ]:
cursor.execute('select count(1) as count, star_rating from {0}.{1} group by star_rating'.format(database_name, raw_table_name))

df = as_pandas(cursor)
df

### Optional - SQL Magic in Jupyter Notebooks

You can use the built-in functionality in Jupyter to create shortcut magic commands to fit your needs. Here we will use the pyathena library like above to query the review data, but this time we will encapsulate the call in the Jupyter magic command.

In [ ]:
import pyathena
from pyathena.util import as_pandas

from IPython.core import magic_arguments
from IPython.core.magic import cell_magic, Magics, magics_class


def query_athena(sql, region_name, s3_staging_dir):
    cursor = pyathena.connect(
        region_name=region_name,
        s3_staging_dir="{}".format(s3_staging_dir)).cursor()
    cursor.execute(sql)
    return cursor

@magics_class
class AthenaMagics(Magics):
    s3_staging_dir = None
    region_name = None

    
    def parse_args(self, line):
        args = magic_arguments.parse_argstring(self.athena, line)

        # s3 staging directory
        if args.s3_staging_dir is None and self.s3_staging_dir is None:
            raise ValueError("s3_staging_dir for Athena should be set")
        if args.s3_staging_dir is not None:
            self.s3_staging_dir = args.s3_staging_dir
 
        # region name
        if args.region_name is None and self.region_name is None:
            raise ValueError("region_name for Athena should be set")
        if args.region_name is not None:
            self.region_name = args.region_name
 
    @cell_magic
    @magic_arguments.magic_arguments()
    @magic_arguments.argument('--s3_staging_dir', '-s',
        help='s3 path required by athena for writing query results (e.g. s3://your/staging/dir)'
    )
    @magic_arguments.argument('--region_name', '-r',
        help='aws region name (e.g. us-west-2)'
    )
    def athena(self, line='', cell=None):
        self.parse_args(line)
        cursor = query_athena(cell, self.region_name, self.s3_staging_dir)
        return as_pandas(cursor)

    
ip = get_ipython()
ip.register_magics(AthenaMagics)

In [ ]:
print(region)
print("s3://{0}/temp".format(bucket))

Now that the magic is registered you can use the `%%athena` magic to query data registered in your Glue Data Catalog.
**Replace the {{region}} and {{bucket/temp}} with the region and bucket variable values above Ex.**

%%athena -r us-west-2 -s s3://escience-workshop/temp

In [ ]:
%%athena -r {{region}} -s {{bucket/temp}}
select * 
from reviews.raw limit 10;

You have successfully completed this lab!!!! 

In this lab you learned to store and retrieve objects with S3, do data manipulation with a number of tools in python like pandas, seaborn, and sklearn, and register and query data in AWS to start building out a dat lake to democratize access to datasets for machine learning.

### Cleanup